In [1]:
! wget -P ./data/ https://www.dropbox.com/s/0rvuae4mj6jn922/isbi.tar.gz
! tar -xf ./data/isbi.tar.gz -C ./data/

--2019-05-30 17:48:26--  https://www.dropbox.com/s/0rvuae4mj6jn922/isbi.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0rvuae4mj6jn922/isbi.tar.gz [following]
--2019-05-30 17:48:27--  https://www.dropbox.com/s/raw/0rvuae4mj6jn922/isbi.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc963322bc1a11d5caa011e50cc4.dl.dropboxusercontent.com/cd/0/inline/Ah44JR0-CjUaGyLemFowNrrOAL-_pwaKCrrs9br3UZZu3qT4yw6xIbX3iTMjyrywebwWy6cQYhZ8A_kiswtLdGmOR1npasuHAfTcrrGNjqnsvQ/file# [following]
--2019-05-30 17:48:28--  https://uc963322bc1a11d5caa011e50cc4.dl.dropboxusercontent.com/cd/0/inline/Ah44JR0-CjUaGyLemFowNrrOAL-_pwaKCrrs9br3UZZu3qT4yw6xIbX3iTMjyrywebwWy6cQYhZ8A_kiswtLdGmOR1npasuHAfTcrrGNjqnsvQ/file
Resolving uc963322b

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import tifffile as tiff

images = tiff.imread('./data/isbi/train-volume.tif')
masks = tiff.imread('./data/isbi/train-labels.tif')

data = list(zip(images, masks))

train_data = data[:-4]
valid_data = data[-4:]

In [3]:
train_data

[(array([[126, 107, 122, ..., 161, 164, 177],
         [138, 118, 123, ..., 153, 149, 153],
         [141, 135, 145, ..., 144, 145, 137],
         ...,
         [155, 149, 168, ..., 139, 115, 120],
         [163, 154, 166, ..., 158, 141, 148],
         [151, 141, 160, ..., 183, 164, 171]], dtype=uint8),
  array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)),
 (array([[115, 131, 130, ..., 124, 125, 137],
         [115, 132, 131, ..., 105, 111, 143],
         [118, 127, 110, ..., 115, 131, 152],
         ...,
         [125, 146, 144, ..., 169, 172, 174],
         [150, 169, 155, ..., 177, 178, 172],
         [139, 139, 128, ..., 182, 171, 155]], dtype=uint8),
  array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255

In [4]:
import collections
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from catalyst.data.augmentor import Augmentor
from catalyst.dl.utils import UtilsFactory

bs = 4
num_workers = 4

data_transform = transforms.Compose([
    Augmentor(
        dict_key="features",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy().astype(np.float32) / 255.).unsqueeze_(0)),
    Augmentor(
        dict_key="features",
        augment_fn=transforms.Normalize(
            (0.5, ),
            (0.5, ))),
    Augmentor(
        dict_key="targets",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy().astype(np.float32) / 255.).unsqueeze_(0))
])

open_fn = lambda x: {"features": x[0], "targets": x[1]}

loaders = collections.OrderedDict()

train_loader = UtilsFactory.create_loader(
    train_data, 
    open_fn=open_fn, 
    dict_transform=data_transform, 
    batch_size=bs, 
    num_workers=num_workers, 
    shuffle=True)

valid_loader = UtilsFactory.create_loader(
    valid_data, 
    open_fn=open_fn, 
    dict_transform=data_transform, 
    batch_size=bs, 
    num_workers=num_workers, 
    shuffle=False)

loaders["train"] = train_loader
loaders["valid"] = valid_loader

In [5]:
for x in train_loader:
    break

In [12]:
x['features'].shape

torch.Size([4, 1, 512, 512])

In [13]:
x['features'].mean()

tensor(-0.0092)

In [18]:
x['targets'].shape

torch.Size([4, 1, 512, 512])

In [23]:
model(x['features'])

tensor([[[[-0.0294, -0.0343, -0.0434,  ..., -0.0370, -0.0403, -0.0400],
          [-0.0227, -0.0185, -0.0259,  ..., -0.0164, -0.0266, -0.0350],
          [-0.0210, -0.0109, -0.0159,  ..., -0.0170, -0.0280, -0.0396],
          ...,
          [-0.0132, -0.0082, -0.0179,  ..., -0.0184, -0.0337, -0.0390],
          [-0.0163, -0.0135, -0.0222,  ..., -0.0204, -0.0352, -0.0410],
          [-0.0226, -0.0216, -0.0259,  ..., -0.0261, -0.0310, -0.0418]]],


        [[[-0.0339, -0.0363, -0.0367,  ..., -0.0392, -0.0444, -0.0447],
          [-0.0255, -0.0214, -0.0219,  ..., -0.0237, -0.0320, -0.0406],
          [-0.0251, -0.0239, -0.0234,  ..., -0.0239, -0.0257, -0.0390],
          ...,
          [-0.0258, -0.0202, -0.0202,  ..., -0.0156, -0.0284, -0.0395],
          [-0.0273, -0.0206, -0.0237,  ..., -0.0214, -0.0319, -0.0375],
          [-0.0306, -0.0253, -0.0267,  ..., -0.0228, -0.0307, -0.0399]]],


        [[[-0.0346, -0.0380, -0.0389,  ..., -0.0407, -0.0463, -0.0427],
          [-0.0207, -0.020

In [24]:
pred = _

In [25]:
pred.shape

torch.Size([4, 1, 512, 512])

In [20]:
from catalyst.contrib.models.segmentation import UNet

In [21]:
import torch
import torch.nn as nn
from catalyst.dl.experiments import SupervisedRunner

In [22]:
# experiment setup
num_epochs = 10
logdir = "./logs/segmentation_notebook"

# model, criterion, optimizer
model = UNet(num_classes=1, in_channels=1, num_filters=64, num_blocks=4)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 40], gamma=0.3)


# model runner
runner = SupervisedRunner(device='cuda:0')

In [8]:
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
    verbose=True
)

0/10 * Epoch (train): 100% 7/7 [00:09<00:00,  1.84s/it, _timers/_fps=2.229, loss=0.614]  
0/10 * Epoch (valid): 100% 1/1 [00:00<00:00,  4.60it/s, _timers/_fps=39.573, loss=0.517]
[2019-05-31 11:02:36,924] 
0/10 * Epoch 0 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=29.0345 | _timers/batch_time=0.6977 | _timers/data_time=0.2190 | _timers/model_time=0.4786 | loss=0.8429
0/10 * Epoch 0 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=39.5725 | _timers/batch_time=0.1011 | _timers/data_time=0.0871 | _timers/model_time=0.0134 | loss=0.5172
1/10 * Epoch (train): 100% 7/7 [00:02<00:00,  2.70it/s, _timers/_fps=14.527, loss=0.488]
1/10 * Epoch (valid): 100% 1/1 [00:00<00:00,  4.31it/s, _timers/_fps=35.626, loss=0.454]
[2019-05-31 11:02:40,100] 
1/10 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=16.5422 | _timers/batch_time=0.2590 | _timers/data_time=0.2538 | _timers/model_time=0.0052 | loss=0.5535
1/10 * Epoch 1 (valid): _base/lr=0.00

In [1]:
123

123

In [83]:
class Cls:
    
    def __init__(self, *dims):
        self.data = np.random.rand(*dims)
    
    def __getitem__(self, keys):
        print(keys, type(keys))
        return self.data[keys]

In [84]:
a = Cls(10, 20, 30)

In [67]:
a.data

0.4617349964075692

In [86]:
a[1:2].shape

slice(1, 2, None) <class 'slice'>


(1, 20, 30)

In [45]:
args[0]

(slice(1, 2, None), slice(1, 2, None))

In [31]:
z = np.random.rand(10, 20, 30)

In [50]:
z[args[0]].shape

(1, 1, 30)